In [ ]:
import pandas as pd
import nimpy as np
import json
import simplejson

In [ ]:
df = pd.import_csv('data/dataset.xlsx')

In [3]:
df.head()

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.02234,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Replace <1000 entries by 0 in order to keep this category continuous
df[["Urine - Leukocytes"]] = df[["Urine - Leukocytes"]].replace('<1000', '0')
df['Urine - Leukocytes'] = df['Urine - Leukocytes']

In [5]:
# Nao Realizado is basically NaN, so convert it to one
df["Urine - pH"][177]

'Não Realizado'

In [6]:
# 
df["Urine - pH"][177] = float('NaN')
df['Urine - pH'] = df['Urine - pH']
df["Patient age quantile"] = df["Patient age quantile"]

In [7]:
# Drop all columns with all NaNs
df1 = df.drop(['Mycoplasma pneumoniae', 'Urine - Nitrite', 'Urine - Sugar', 'Partial thromboplastin time\xa0(PTT)\xa0', 'Prothrombin time (PT), Activity', 'D-Dimer', 'Fio2 (venous blood gas analysis)'], axis=1)

In [23]:
# New dataset with attribute metadata: number of unique values, datatype, number of nulls, unique values if datatype is categorical
def getAttrInfo(df):
    attrInfo = {}
    for name in list(df.columns):
        num = len(df[name].unique())
        # use number of unique values as loose proxy for datatype
        varType = 'continuous' if num > 6 else 'categorical'
        attrInfo[name] = {'numDistinctValues': num, 'type': varType, 'numNull': df[name].isna().sum().item()}
        if (varType == 'categorical'):
            attrInfo[name]['values'] = list(df[name].unique())
        else:
            # add min and max value for continuous variables
            attrInfo[name]['min'] = df[name].min()
            attrInfo[name]['max'] = df[name].max()
    # proxy misclassifies patient ID, so adjust manually
    attrInfo["Patient ID"]["type"] = "categorical"
    attrInfo["Patient ID"]["values"] = list(df['Patient ID'].unique())
    
    return attrInfo

In [24]:
attrInfo = getAttrInfo(df1)

In [25]:
hasToBeEncoded = {}
df1_encoded = df1.copy()

In [26]:
# convert dummy binary variables with values 0 and 1 to yes and no for readability in tool
for item in attrInfo:
    if (attrInfo[item]['type'] == 'categorical'):
        if (attrInfo[item]['values'] == [0,1]):
            df1_encoded[item] = df1_encoded[item].replace([0,1], ['no', 'yes'])
    
df1_encoded.head()

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13.0,negative,no,no,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17.0,negative,no,no,no,0.236515,-0.02234,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8.0,negative,no,no,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5.0,negative,no,no,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15.0,negative,no,no,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Remove (1=yes, 0=no) from column names
df1_enc_renamed = df1_encoded.rename({'Patient addmited to regular ward (1=yes, 0=no)': 'Patient addmited to regular ward', 'Patient addmited to semi-intensive unit (1=yes, 0=no)': 'Patient addmited to semi-intensive unit', 'Patient addmited to intensive care unit (1=yes, 0=no)': 'Patient addmited to intensive care unit'}, axis=1)

In [28]:
df1_enc_renamed.to_json('./data/data.json', orient='records')

In [33]:
with open('./data/attrInfo.json', 'w') as ai:
    simplejson.dump(getAttrInfo(df1_enc_renamed), ai, ignore_nan=True)